<a href="https://colab.research.google.com/github/Avirup-Gupta/MDC-CNN/blob/main/MajorityVoting_MDC_CNN_FS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Some cells are marked as #CHECK, ar for checking purposes. Some marked as #NOT TO BE EXECUTED should be avoided if not absolutely required 

In [1]:
!pip install mne
!pip install PyIF
!pip install nose
import nose
import mne
import numpy as np
import os
import pandas as pd
import collections
from matplotlib import pyplot as plt
import cv2
from sklearn import preprocessing

     |████████████████████████████████| 7.0MB 7.5MB/s 
     |████████████████████████████████| 163kB 7.6MB/s 


In [2]:
######### MAIN CODE BEGINS ##############
xlsx_file_name='/content/drive/MyDrive/TUH data/seizures_v36r.xlsx'

In [3]:
excel_file = os.path.join(xlsx_file_name)
file_list = pd.read_excel(excel_file, sheet_name='dev', usecols=['Filename'])
patient_id = pd.read_excel(excel_file, sheet_name='dev', usecols=['Patient'])
seizure_type = pd.read_excel(excel_file, sheet_name='dev', usecols=['Seizure Type'])
start_time=pd.read_excel(excel_file, sheet_name='dev', usecols=['Start'])
start_time=(start_time.to_numpy())
stop_time=pd.read_excel(excel_file, sheet_name='dev', usecols=['Stop'])
stop_time=(stop_time.to_numpy())
file_list=np.array(file_list)
patient_id=np.array(patient_id)
seizure_type=np.array(seizure_type)


In [ ]:
#CHECK
no=104 ##input the index 
no=no-2 ##adjusting with the seizure file list xlsx
print('file_no:', no)
print(file_list[no])
print('seizure type:',seizure_type[no])
print('start time',start_time[no])
print('stop time',stop_time[no])

In [4]:
fpath=['intial']
labels=[1]
labels=np.array([labels])

#in our case the initial file was a non seizure one so we initialized according, can be changed.
#the multiplication of sampling freq must be considered so as to not exceed the length 
cl_st=[10]
cl_en=[50]


loc='/content/drive/MyDrive/TUH data' #for google colab
#loc='C:/TUH/v1.1.0/edf' #for PC

for i in range(1,1407):
  if seizure_type[i]==['TCSZ'] or seizure_type[i]==['FNSZ'] or seizure_type[i]==['CPSZ'] or seizure_type[i]==['GNSZ']:
    fl=(file_list[i])
    fl=fl[0]
    #print('Original path: ',fl)
    #print('The file path after stripping: ',fl[6:])
    fl=fl.replace('.tse','.edf')
    s=os.path.join(loc,(fl[6:]))
    #print('The modified path is: ',s) 
    #s=fl.replace(fl[:fl.index[6]],'/content/drive/MyDrive/TUH data')
    if os.path.isfile(s)==True:
      
      fpath.append(s)
      labels=np.append(labels, [[1]], axis=0)
      check_neg=start_time[i]-20
      if check_neg>0:
        cl_st.append(float(start_time[i]-20))
      else:
        cl_st.append(0)  
      cl_en.append(float(stop_time[i]))

    
  else:
    fl=(file_list[i])
    fl=fl[0]
    #print('Original path: ',fl)
    #print('The file path after stripping: ',fl[6:])
    fl=fl.replace('.tse','.edf')
    s=os.path.join(loc,(fl[6:]))
    #print('The modified path is: ',s)
    #fpath.append(str(file_list[i]).replace('.tse','.edf'))
    if os.path.isfile(s)==True:
      
      fpath.append(s)
      labels=np.append(labels, [[0]], axis=0)
      cl_st.append(10)
      cl_en.append(50)

    

fpath=fpath[1:]
labels=labels[1:]
cl_st=cl_st[1:]
cl_en=cl_en[1:]

#print('The modified paths are:',fpath)
print('size of fpath:', len(fpath))
#print('The modified labels are:',labels)
train_labels=labels
print(cl_st)
print(cl_en)

size of fpath: 68
[10, 10, 10, 10, 0, 0, 0, 187.9101, 10, 10, 14.055100000000003, 67.3375, 0, 10, 26.273400000000002, 99.1406, 146.8203, 179.3398, 229.0547, 277.7539, 324.8438, 397.4219, 450.0469, 500.55859999999996, 579.1484, 622.6211, 668.1211, 709.4531, 751.1211, 807.1211, 848.375, 891.1719, 933.0547, 982.0273, 1033.3477, 1074.7461, 1111.9414, 1159.0664, 1206.3477, 1267.9453, 1311.6914, 1352.2188, 1393.9688, 1471.8203, 1531.1211, 1568.793, 1627.0195, 1660.9453, 1704.3281, 1753.2461, 1790.8203, 1839.2188, 1886.8203, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
[50, 50, 50, 50, 173.3791, 175.5054, 150.5157, 308.0, 50, 50, 162.3118, 223.0086, 154.51, 50, 52.0742, 125.8398, 172.9219, 208.5391, 256.3516, 303.1523, 351.4453, 425.1211, 477.1484, 527.5586, 605.6484, 649.5195, 695.4219, 736.5547, 779.2188, 833.6211, 875.9727, 918.0742, 960.5547, 1008.2266, 1058.6484, 1099.9453, 1138.8398, 1186.0664, 1232.1484, 1293.0469, 1336.5938, 1378.1211, 1420.0664, 1497.0195, 1556.0195, 1

In [ ]:
#CHECK
print(fpath[53])
cl_st[53]
cl_en[53]

/content/drive/MyDrive/TUH data/01_tcp_ar/046/00004671/s008_2012_08_24/00004671_s008_t000.edf


50

In [5]:
### Sampling frequency part ###

def convert(lst): 
    return (lst.split('\t')) 

tse_file_path=[]
fs=[]
for f in fpath:
  fa=f.replace('TUH data', 'seizure/tse')
  fa=fa.replace('.edf','.tse')
  annot_file_path=str(fa)
  print('Path exists: ',os.path.exists(annot_file_path))
  l=len(annot_file_path)
  print('The length of the path is: ',l)
  c=annot_file_path[l-2]
  #print('the l-2 is:',c)
  tse_file_path.append(annot_file_path)
  '''  
  if c=='x':
    print('The file is already converted to a text file')
  else:
    print('Converting the TSE file to a txt file')
    annot_file_path1=annot_file_path.replace('.tse','.txt')
    print('The converted file path is: ',annot_file_path1)
    #os.rename(annot_file_path,annot_file_path1)
    annot_file_path=annot_file_path1 
  '''
  f1 = open(annot_file_path,'r+')
  string_list=f1.readlines() #when using f.read() before f.readlines() the string_list generated due to readlines is shown blank
  print(string_list[6])
  annot=string_list[6]
  f1.close()

  con_annot=convert(annot)
  st_time=float(con_annot[0])
  en_time=float(con_annot[1])
  type_of_seizure=con_annot[2]
  duration=en_time-st_time

  raw = mne.io.read_raw_edf(f, verbose=False)
  channel_data=raw.get_data()
  print('The shape of the data:',channel_data.shape)
  sampling_freq=channel_data.shape[1]/duration
  print('The sampling frequency of the file:',sampling_freq)
  print('***********************************')
  fs.append(sampling_freq)

Path exists:  True
The length of the path is:  96
0.0000	20.0000	bckg	0.985839

The shape of the data: (32, 8000)
The sampling frequency of the file: 400.0
***********************************
Path exists:  True
The length of the path is:  96
0.0000	272.0000	0.301196

The shape of the data: (32, 108800)
The sampling frequency of the file: 400.0
***********************************
Path exists:  True
The length of the path is:  96
0.0000	234.0000	bckg	0.594640

The shape of the data: (32, 93600)
The sampling frequency of the file: 400.0
***********************************
Path exists:  True
The length of the path is:  96
0.0000	224.0000	bckg	0.386299

The shape of the data: (32, 89600)
The sampling frequency of the file: 400.0
***********************************
Path exists:  True
The length of the path is:  96
0.0000	203.0000	bckg	0.229591

The shape of the data: (32, 81200)
The sampling frequency of the file: 400.0
***********************************
Path exists:  True
The length of the

In [ ]:
#CHECK
print(fs)

[400.0, 400.0, 400.0, 400.0, 400.0, 400.0, 400.0, 400.0, 400.0, 400.0, 400.0, 400.0, 400.0, 400.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 250.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0, 256.0]


In [ ]:
#NOT TO BE EXECUTED IF ALREADY THE TSE FILES ARE MODIFIED#
'''
TSE file modification
Some TUH-eeg annotation files(.tse files) are in a different format from the majority of the files
The Sampling Freqency Part calculates the FS(sampling frequency) from the TSE files, hence before that we need to bring all to a common format
This portion of the code is meant for modification, only to be executed if files are not modified already. Please note that modifications can
also be made manually 

'''
for i in fs:
  if i!=float(256) or i!=float(400) or i!=float(250):
    ii=fs.index(i)
    fp=tse_file_path[ii]
    f1 = open(fp,'r+')
    string_list=f1.readlines() #when using f.read() before f.readlines() the string_list generated due to readlines is shown blank
    
    annot1=string_list[6]
    annot2=string_list[8] 
    f1.close()

    con_annot1=convert(annot1)
    con_annot2=convert(annot2)
    st_time=float(con_annot1[0])
    en_time=float(con_annot2[1])
    duration=en_time-st_time

    raw = mne.io.read_raw_edf(fpath[ii], verbose=False)
    channel_data=raw.get_data()
    
    sampling_freq=channel_data.shape[1]/duration
    fs[ii]=sampling_freq 

In [ ]:
#CHECK
print(np.sum(np.array(train_labels)))

In [ ]:
#CHECK
print(np.transpose(np.array(fpath[1:])))
print(np.transpose(labels))
print(cl_st)
print(cl_en)

#print(labels[1:])

Next there are some algorithms mentioned, PCC, PLV, TE. By default PLV and TE are chosen and accordingly subsequent outputs will be fed to the MULTI CNN. Run only those which are required.

In [6]:
### FOR BANDPASS Filter##
'Enter the upper and lower frequncy of corresponding band '
l_frequency=int(input('Enter the lower cutoff frequency: '))
h_frequency=int(input('Enter the higher cutoff frequency: '))

Enter the lower cutoff frequency: 1
Enter the higher cutoff frequency: 100


In [7]:
## PLV ##
import numpy as np
import scipy.signal as sig

def hilphase(y1,y2):
    sig1_hill=sig.hilbert(y1)
    sig2_hill=sig.hilbert(y2)
    pdt=(np.inner(sig1_hill,np.conj(sig2_hill))/(np.sqrt(np.inner(sig1_hill,
               np.conj(sig1_hill))*np.inner(sig2_hill,np.conj(sig2_hill)))))
    phase = np.angle(pdt)

    return phase

train_data_plv=[]
ii=0
for f in fpath:
  print(f, ii+1)
  raw = mne.io.read_raw_edf(f, verbose=False)
  ##
  raw.load_data()
  
  raw=raw.filter(l_freq=l_frequency, h_freq=h_frequency)
  ##
  channel_data=raw.get_data()
  #print(np.all(np.isnan(channel_data)))
  #print(np.all(np.isinf(channel_data)))
  channel_data=channel_data[:, int(cl_st[ii]*fs[ii]):int(cl_en[ii]*fs[ii])]
  print('Starting and Ending point respectively: ',int(cl_st[ii]*fs[ii]),int(cl_en[ii]*fs[ii]))
  sh=channel_data.shape[0]
  ii=ii+1
  
  s=(sh, sh)
  plv2=np.zeros(s)
  ch_ls=[]
  for i in range(0, sh):
    for j in range(0,sh):
      plv2[i][j]=hilphase(channel_data[i], channel_data[j])
      if (np.any(np.isnan(plv2[i][j])))==True:
        plv2[i][j]=0
      if (np.any(np.isinf(plv2[i][j])))==True:
        plv2[i][j]=1 

  #np.append(train_data,pcc2, axis=0)
  plv2=cv2.resize(plv2, (32,32))
  
  plv2=preprocessing.normalize(plv2)#additional normalization layer checks whether any data is NaN
  train_data_plv.append(plv2)

/content/drive/MyDrive/TUH data/01_tcp_ar/002/00000258/s002_2003_07_21/00000258_s002_t000.edf 1
Starting and Ending point respectively:  4000 20000
/content/drive/MyDrive/TUH data/01_tcp_ar/002/00000258/s002_2003_07_21/00000258_s002_t002.edf 2
Starting and Ending point respectively:  4000 20000
/content/drive/MyDrive/TUH data/01_tcp_ar/002/00000258/s003_2003_07_22/00000258_s003_t000.edf 3
Starting and Ending point respectively:  4000 20000
/content/drive/MyDrive/TUH data/01_tcp_ar/002/00000258/s003_2003_07_22/00000258_s003_t001.edf 4
Starting and Ending point respectively:  4000 20000
/content/drive/MyDrive/TUH data/01_tcp_ar/002/00000258/s003_2003_07_22/00000258_s003_t002.edf 5
Starting and Ending point respectively:  0 69351
/content/drive/MyDrive/TUH data/01_tcp_ar/002/00000258/s003_2003_07_22/00000258_s003_t003.edf 6
Starting and Ending point respectively:  0 70202
/content/drive/MyDrive/TUH data/01_tcp_ar/002/00000258/s003_2003_07_22/00000258_s003_t004.edf 7
Starting and Ending po

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in cdouble_scalars
  if __name__ == '__main__':


/content/drive/MyDrive/TUH data/01_tcp_ar/046/00004671/s007_2012_08_04/00004671_s007_t000.edf 16
Starting and Ending point respectively:  25379 32214


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in cdouble_scalars
  if __name__ == '__main__':


/content/drive/MyDrive/TUH data/01_tcp_ar/046/00004671/s007_2012_08_04/00004671_s007_t000.edf 17
Starting and Ending point respectively:  37585 44268


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in cdouble_scalars
  if __name__ == '__main__':


/content/drive/MyDrive/TUH data/01_tcp_ar/046/00004671/s011_2013_07_08/00004671_s011_t008.edf 66
Starting and Ending point respectively:  2560 12800


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in cdouble_scalars
  if __name__ == '__main__':


/content/drive/MyDrive/TUH data/01_tcp_ar/046/00004671/s011_2013_07_08/00004671_s011_t009.edf 67
Starting and Ending point respectively:  2560 12800
/content/drive/MyDrive/TUH data/01_tcp_ar/046/00004671/s011_2013_07_08/00004671_s011_t010.edf 68
Starting and Ending point respectively:  2560 12800


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in cdouble_scalars
  if __name__ == '__main__':


In [ ]:
## TE ##
from PyIF import te_compute as te

train_data_te=[]
ii=0
for f in fpath:
  print(f, ii+1)
  raw = mne.io.read_raw_edf(f, verbose=False)
  ##
  raw.load_data()
  
  raw=raw.filter(l_freq=l_frequency, h_freq=h_frequency)
  ##
  
  channel_data=raw.get_data()
  channel_data=channel_data[:, int(cl_st[ii]*fs[ii]):int(cl_en[ii]*fs[ii])]
  sh=channel_data.shape[0]
  ii=ii+1
  s=(sh, sh)
  te2=np.zeros(s)
  ch_ls=[]
  for i in range(0, sh):
    for j in range(0,sh):
      te2[i][j]=te.te_compute(channel_data[i], channel_data[j], k=1, embedding =1, GPU=False)
      if (np.any(np.isnan(te2[i][j])))==True:
        te2[i][j]=0
      if (np.any(np.isinf(te2[i][j])))==True:
        te2[i][j]=1  

  #np.append(train_data,pcc2, axis=0)
  te2=cv2.resize(te2, (32,32))
  te2=preprocessing.normalize(te2)#additional normalization layer checks whether any data is NaN
  train_data_te.append(te2)

In [ ]:
######### CNN SECTION ##########

Next is the CNN section. Cells are marked separately for PLV and TE

In [8]:
#for PLV
train_data_plv=np.array(train_data_plv)

#assert not np.any(np.isnan(train_data))
print('The shape of the train data', train_data_plv.shape)

The shape of the train data (68, 32, 32)


In [ ]:
#for TE
train_data_te=np.array(train_data_te)

#assert not np.any(np.isnan(train_data))
print('The shape of the train data', train_data_te.shape)

In [ ]:
#CHECK
#optional for visualizing the data
t=np.size(train_data, 0) 
for x in range(0,t):
  print('The image of the correlation of file', x+1)
  plt.subplot(1, t, x+1)
  plt.imshow(train_data[x])
  plt.title('Image:'+ str(x+1))

plt.show() 
#plt.imshow(train_data[5])

plt.plot(train_data[5])

In [9]:
import tensorflow as tf
from tensorflow import keras
from sklearn import preprocessing
import numpy as np
from keras.utils import np_utils
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import concatenate, Input  
from keras.layers.core import Dense, Flatten, Activation, Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.datasets import cifar10
from keras.optimizers import Adam, RMSprop
import matplotlib.pyplot as plt
import cv2

In [10]:
class LeNet:
  @staticmethod
  def build(input_shape, classes):
    model=Sequential()
    model.add(BatchNormalization(input_shape=input_shape))
    #First convolution
    model.add(Conv2D(32, kernel_size=9, padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(32, kernel_size=7, padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    #Second convolution
    model.add(Conv2D(64, kernel_size=9, padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, kernel_size=7, padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
   
    #Flattening
    model.add(Flatten())
    model.add(Dense(500))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    #Classifier
    model.add(Dense(classes))
    model.add(Activation('sigmoid'))
    model.summary()
    
    return model

In [11]:
#For PLV generates x_train1
img_rows=32
img_cols=32
img_channels=1
input_shape=(img_rows, img_cols, img_channels)
classes=2

x_train1=train_data_plv.astype('float32')

y_train=train_labels
y_train=np_utils.to_categorical(y_train, classes)

#x_train=x_train
x_train1=np.expand_dims(x_train1, axis=3)

print('The shape of the train_data: ', x_train1.shape)

The shape of the train_data:  (68, 32, 32, 1)


In [ ]:
#For TE generates x_train2
img_rows=32
img_cols=32
img_channels=1
input_shape=(img_rows, img_cols, img_channels)
classes=2

x_train2=train_data_te.astype('float32')

y_train=train_labels
y_train=np_utils.to_categorical(y_train, classes)

x_train2=np.expand_dims(x_train2, axis=3)

print('The shape of the train_data: ', x_train2.shape)

In [12]:
## IF ONLY USING PLV RUN THIS PART ## SIMILARLY TE CAN ALSO BE USED BUT PLV IS FASTER
x_train2=x_train1


In [14]:
first=LeNet.build(input_shape, classes)

first.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy', 'TruePositives', 'FalseNegatives', 'TrueNegatives', 'FalsePositives'])
history1=first.fit(x_train1, y_train, validation_split=0.1, batch_size=15, epochs=20, verbose=1)

second=LeNet.build(input_shape, classes)

second.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy', 'TruePositives', 'FalseNegatives', 'TrueNegatives', 'FalsePositives'])
history2=second.fit(x_train2, y_train, validation_split=0.1, batch_size=15, epochs=20, verbose=1)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 32, 32, 1)         4         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 32)        2624      
_________________________________________________________________
activation_6 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 32)        50208     
_________________________________________________________________
activation_7 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 16, 16, 64)       

In [15]:
x_test1=x_train1
x_test2=x_train2

y_test=y_train

In [22]:
def predictions(model, x_test):
  yhats=model.predict(x_test)
  yhats=np.array(yhats)

  return yhats

In [23]:
yhats1=predictions(first, x_test1)
yhats2=predictions(second, x_test2)

In [ ]:
#CHECK
print('Predictions of the 1st model', yhats1)
print('Predictions of the 2nd model', yhats2)

In [29]:
i1=np.add(yhats1, yhats2)
predicted_labels=np.array(np.argmax(i1, axis=1))

In [ ]:
#CHECK
print(yhats1[1],yhats2[1],i1[1])

In [37]:
print('predicted_labels:',predicted_labels)
label_test=np.transpose(np.array(labels))
label_test=label_test[0] #Converting from [[labels]] to[labels] 
print('labels:', label_test)


[0.93892825 0.06810161] [0.9959717  0.01570031] [1.9348999  0.08380193]
predicted_labels: [0 0 0 0 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
labels: [0 0 0 0 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [42]:
correct=0
incorrect=0
for i in range(0,np.size(label_test)):
  if predicted_labels[i]==label_test[i]:
    correct=correct+1

  else:
    incorrect=incorrect+1

print('The Prediction accuracy of the model: ', correct/np.size(label_test))

The Prediction accuracy of the model:  1.0
